<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8cb7b9a98a5d52d2e4f59fd5f1350da086be07f520960043e889971ac89661f2
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 14:24:46
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -28.86 K (-0.2%)
Current PnL: -19.82 L (-13.21%)
CY Booked + Current PnL: -7.72 L (-5.15%)
-------------------
Total profit:  2.17 L
Total loss:  -22.00 L
-------------------
Total Booked + Current PnL: 19.26 L (15.64%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.53%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.01 L (61.34%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
85      VOLTAS   1530.0     3.26    61.0       X-MC     2.81  216645.0   
58    RELIANCE   1533.0    -9.53    66.0       X-LC     3.17  226091.0   
2   ABBOTINDIA  35195.0    -9.24    43.0       X-MC     3.32   89730.0   
78  TTKPRESTIG    770.0    96.16    49.0       M-SC     1.97   84502.0   
42         ITC    452.0   -37.08    64.0       X-LC     1.96  203595.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
85      24903.0  12847.0        0.92         12.99   5.93  19.69     99.0   
58      15725.0  13068.0        0.06          7.48   5.78  13.68     37.0   
2        -948.0  15855.0       -0.68         -1.05  17.67  16.44    101.0   
78     -16275.0  16368.0        0.50        -16.15  19.37   0.09    245.0   
42       3457.0  17428.0        0.10          1.73   8.56  10.44      4.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
85     1.94        1.53    20.67     XY25      NTT          AC  
58     1.20        1.60    25.23     XY25      NTT  REFINERIES  
2     -0.06        0.64    18.92      X40      ATH      PHARMA  
78    -0.99        0.60    11.02    OX40N      NTT    DURABLES  
42     0.20        1.44     7.39      X40      NTT        FMCG

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
62  SAMMAANCAP   326.0  -197.73    81.0       M-SC    24.60  169092.0   
70    SURYODAY   240.0    68.86    64.0       H-SC    12.26  155856.0   
82  VAIBHAVGBL   521.0    54.82    61.0       H-SC     4.41  134429.0   
36         IEX   219.0   -30.16    62.0       H-SC    17.42  199493.0   
73   TATAELXSI  9161.0   -22.20    55.0       H-MC     7.49  105174.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
62      19872.0  124316.0        7.73         13.32   73.52  96.62    208.0   
70     -23215.0   88947.0        2.68        -12.96   57.07  36.71    135.0   
82     -48346.0  159419.0        1.76        -26.45  118.59  60.77    125.0   
36       -461.0   97692.0        1.72         -0.23   48.97  48.63    115.0   
73     -26661.0   68889.0        1.18        -20.22   65.50  32.03     98.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
62     0.16        1.20    82.99     XY25      NTT    FINANCE  
70    -0.26        1.10    54.64       XR      NTT      BANKS  
82    -0.30        0.95    20.40       XR      NTT  JEWELLERY  
36    -0.00        1.41    11.30       XR      NTT       MISC  
73    -0.39        0.74    17.80    OX40N      NTT         IT

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
31  HINDUNILVR  2922.00   -10.50    44.0       X-LC     7.66  251360.0   
26  GREENPANEL   537.00   226.23    51.0       M-SC     5.31  145717.0   
7   ASIANTILES   137.00  7055.56    62.0       L-SC     7.19   78484.0   
21      COLPAL  3767.14    -4.54    45.0       X-MC     8.16  223040.0   
86   WHIRLPOOL  2270.00   -39.28    63.0       M-SC     3.86  102735.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
31      -5703.0   40846.0       -3.38         -2.22   16.25  13.67     24.0   
26     -37361.0  116340.0       -2.93        -20.41   79.84  43.14    230.0   
7      -15326.0   91944.0       -2.67        -16.34  117.15  81.67    269.0   
21     -40325.0  153675.0       -2.47        -15.31   68.90  43.04     84.0   
86      11236.0   67517.0       -2.12         12.28   65.72  86.07    223.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
31    -0.14        1.78    17.74     XY25      NTT      FMCG  
26    -0.32        1.03    37.07     XY24      NTT      MISC  
7     -0.17        0.56    51.99       XR      NTT  CERAMICS  
21    -0.26        1.58     3.55     XY25      ATH      FMCG  
86     0.17        0.73    49.19       XR      NTT  DURABLES

In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  Current P/L  \
61    SAIL  228.0    41.94    47.0       M-MC    10.00  223464.0      -1498.0   
36     IEX  219.0   -30.16    62.0       H-SC    17.42  199493.0       -461.0   
56  RECLTD  446.0    45.36    46.0       M-LC     5.70  205012.0       1842.0   
87   WIPRO  420.0   -14.25    46.0       M-LC     5.95  150375.0       -570.0   

     FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  RRR Ind  \
61  169162.0        0.33         -0.67  75.70  74.53    192.0    -0.01   
36   97692.0        1.72         -0.23  48.97  48.63    115.0    -0.00   
56   40285.0       -0.57          0.91  19.65  20.74     55.0     0.05   
87  110029.0       -0.72         -0.38  73.17  72.51     53.0    -0.01   

    CurrAlloc%  Gained% Criteria Strategy Category  
61        1.58    31.37     XY24      BTT    STEEL  
36        1.41    11.30       XR      NTT     MISC  
56        1.45     6.50     XY25      NTT  FINANCE  
87        1.06     5.62       XR      NTT       IT

In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
79  UJJIVANSFB    60.0   122.87    69.0       H-SC    15.26  140508.0   
45   KANSAINER   340.0   -66.87    58.0       H-SC     2.37  227988.0   
78  TTKPRESTIG   770.0    96.16    49.0       M-SC     1.97   84502.0   
73   TATAELXSI  9161.0   -22.20    55.0       H-MC     7.49  105174.0   
18        CERA  9475.0   -23.39    46.0       H-SC     2.62  139725.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
79      -1971.0  21498.0        0.27         -1.38  15.30  13.70    163.0   
45     -41679.0  78017.0       -0.83        -15.46  34.22  13.47    138.0   
78     -16275.0  16368.0        0.50        -16.15  19.37   0.09    245.0   
73     -26661.0  68889.0        1.18        -20.22  65.50  32.03     98.0   
18     -36178.0  78204.0        0.88        -20.57  55.97  23.89    149.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
79    -0.09        0.99    65.84    OX40N      NTT     BANKS  
45    -0.53        1.61    15.93     XY24      NTT    PAINTS  
78    -0.99        0.60    11.02    OX40N      NTT  DURABLES  
73    -0.39        0.74    17.80    OX40N      NTT        IT  
18    -0.46        0.99    20.67    OX40N      NTT  CERAMICS

In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Empty DataFrame
Columns: [Symbol, FTT, Dev%_PE, RSI_14, Conviction, Spread%, Current, Current P/L, FTT Amt, Today P/L%, Current P/L%, FTT%, OTT%, CumlRnk, RRR Ind, CurrAlloc%, Gained%, Criteria, Strategy, Category]
Index: []

In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
56     RECLTD  446.00    45.36    46.0       M-LC     5.70  205012.0   
32   HINDZINC  730.22    29.80    53.0       M-LC    11.00  211867.0   
46   KPIGREEN  731.05     7.10    67.0       H-SC     5.19  127789.0   
80  UNIONBANK  163.00   -11.63    58.0       M-LC     9.09  161079.0   
12  BANKINDIA  190.00   -20.50    67.0       H-MC    14.24  205385.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
56       1842.0   40285.0       -0.57          0.91  19.65  20.74     55.0   
32       6791.0  105785.0        1.02          3.31  49.93  54.89     52.0   
46       2492.0   56432.0       -0.30          1.99  44.16  47.03    141.0   
80      20239.0   24258.0       -0.83         14.37  15.06  31.59     66.0   
12      25577.0   86837.0       -1.47         14.22  42.28  62.52     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
56     0.05        1.45     6.50     XY25      NTT  FINANCE  
32     0.06        1.50    28.20      X5K      ATH   METALS  
46     0.04        0.90    60.00       MH      ATH    POWER  
80     0.83        1.14    44.66     XY24      NTT    BANKS  
12     0.29        1.45    53.16       XR      NTT    BANKS

In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12  BANKINDIA   190.00   -20.50    67.0       H-MC    14.24  205385.0   
86  WHIRLPOOL  2270.00   -39.28    63.0       M-SC     3.86  102735.0   
32   HINDZINC   730.22    29.80    53.0       M-LC    11.00  211867.0   
46   KPIGREEN   731.05     7.10    67.0       H-SC     5.19  127789.0   
37  INDIAMART  4810.62   -55.20    44.0       H-SC     7.16  125181.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
12      25577.0   86837.0       -1.47         14.22   42.28   62.52     88.0   
86      11236.0   67517.0       -2.12         12.28   65.72   86.07    223.0   
32       6791.0  105785.0        1.02          3.31   49.93   54.89     52.0   
46       2492.0   56432.0       -0.30          1.99   44.16   47.03    141.0   
37       1845.0  129788.0        0.52          1.50  103.68  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
12     0.29        1.45    53.16       XR      NTT     BANKS  
86     0.17        0.73    49.19       XR      NTT  DURABLES  
32     0.06        1.50    28.20      X5K      ATH    METALS  
46     0.04        0.90    60.00       MH      ATH     POWER  
37     0.01        0.89    25.22       AR      ATH      MISC

In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
74  TATAMOTORS   1065.00   -53.17    16.0       X-LC    13.41  147305.0   
24       DMART   5391.45   -17.39    33.0       X-LC    10.16  101026.0   
53       QUESS    986.00   -48.76    33.0       X-SC    38.78   50928.0   
51     PAGEIND  51769.93   -27.54    39.0       X-MC     6.80   81840.0   
52        PGHH  17907.65   -29.62    41.0       X-MC     3.22  203595.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
74    -129218.0  239282.0        0.98        -46.73  162.44   39.81     54.0   
24      -1560.0   28368.0       -1.37         -1.52   28.08   26.13     38.0   
53     -14078.0  165002.0       -1.04        -21.66  323.99  232.18    198.0   
51      -1670.0   21696.0       -1.30         -2.00   26.51   23.98     82.0   
52       2775.0   65028.0       -0.96          1.38   31.94   33.76     80.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
74    -0.54        1.04     3.82     XY24      NTT      AUTO  
24    -0.05        0.72    23.66     X40N      ATH      FMCG  
53    -0.09        0.36     0.00     XY24      NTT      MISC  
51    -0.08        0.58     3.70      X40      ATH  APPARELS  
52     0.04        1.44     6.65      X40      ATH      FMCG

In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
66   SIEMENS  4671.50    -2.40    48.0       H-LC     0.79  156915.0   
42       ITC   452.00   -37.08    64.0       X-LC     1.96  203595.0   
85    VOLTAS  1530.00     3.26    61.0       X-MC     2.81  216645.0   
29   HAVELLS  2069.16    -5.90    47.0       X-MC     2.81  233438.0   
58  RELIANCE  1533.00    -9.53    66.0       X-LC     3.17  226091.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
66     -29180.0  76653.0       -0.37        -15.68  48.85  25.51     15.0   
42       3457.0  17428.0        0.10          1.73   8.56  10.44      4.0   
85      24903.0  12847.0        0.92         12.99   5.93  19.69     99.0   
29     -14293.0  89360.0        0.83         -5.77  38.28  30.30     92.0   
58      15725.0  13068.0        0.06          7.48   5.78  13.68     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
66    -0.38        1.11    15.63       AR      ATH  ELECTRICAL  
42     0.20        1.44     7.39      X40      NTT        FMCG  
85     1.94        1.53    20.67     XY25      NTT          AC  
29    -0.16        1.65     6.76      X40      ATH  ELECTRICAL  
58     1.20        1.60    25.23     XY25      NTT  REFINERIES

In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
53       QUESS    986.00   -48.76    33.0       X-SC    38.78   50928.0   
21      COLPAL   3767.14    -4.54    45.0       X-MC     8.16  223040.0   
51     PAGEIND  51769.93   -27.54    39.0       X-MC     6.80   81840.0   
74  TATAMOTORS   1065.00   -53.17    16.0       X-LC    13.41  147305.0   
3          ACC   3906.00   -37.64    51.0       X-MC     2.89  185450.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
53     -14078.0  165002.0       -1.04        -21.66  323.99  232.18    198.0   
21     -40325.0  153675.0       -2.47        -15.31   68.90   43.04     84.0   
51      -1670.0   21696.0       -1.30         -2.00   26.51   23.98     82.0   
74    -129218.0  239282.0        0.98        -46.73  162.44   39.81     54.0   
3      -52501.0  205145.0       -0.06        -22.06  110.62   64.15    174.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
53    -0.09        0.36     0.00     XY24      NTT      MISC  
21    -0.26        1.58     3.55     XY25      ATH      FMCG  
51    -0.08        0.58     3.70      X40      ATH  APPARELS  
74    -0.54        1.04     3.82     XY24      NTT      AUTO  
3     -0.26        1.31     4.03     XY24      BTT    CEMENT

In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%  Current  \
53       QUESS    986.00   -48.76    33.0       X-SC    38.78  50928.0   
51     PAGEIND  51769.93   -27.54    39.0       X-MC     6.80  81840.0   
57      RELAXO   1176.00   -39.30    45.0       X-SC     4.68  82786.0   
2   ABBOTINDIA  35195.00    -9.24    43.0       X-MC     3.32  89730.0   
13   BATAINDIA   2096.00   -35.44    46.0       X-SC     4.45  96454.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
53     -14078.0  165002.0       -1.04        -21.66  323.99  232.18    198.0   
51      -1670.0   21696.0       -1.30         -2.00   26.51   23.98     82.0   
57     -62374.0  138302.0       -0.77        -42.97  167.06   52.31    136.0   
2        -948.0   15855.0       -0.68         -1.05   17.67   16.44    101.0   
13     -32216.0   77510.0       -0.47        -25.04   80.36   35.20    219.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
53    -0.09        0.36     0.00     XY24      NTT      MISC  
51    -0.08        0.58     3.70      X40      ATH  APPARELS  
57    -0.45        0.59    10.81     X40N      NTT  FOOTWEAR  
2     -0.06        0.64    18.92      X40      ATH    PHARMA  
13    -0.42        0.68    10.71      X40      NTT  FOOTWEAR

In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
75    TCS  4389.96   -27.72    57.0       X-LC    12.35  288110.0   
40   INFY  2275.00   -16.73    57.0       X-LC     7.19  323717.0   
42    ITC   452.00   -37.08    64.0       X-LC     1.96  203595.0   
84    VBL   671.64   -18.39    52.0       X-LC     9.88  293799.0   
1     ABB  7934.00   -40.45    47.0       H-LC     6.32  248808.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
75     -57846.0  124550.0       -0.27        -16.72  43.23  19.28      1.0   
40      10829.0  160855.0       -0.57          3.46  49.69  54.87      3.0   
42       3457.0   17428.0        0.10          1.73   8.56  10.44      4.0   
84     -22043.0  134707.0       -1.08         -6.98  45.85  35.67      5.0   
1      -12811.0  132018.0        0.04         -4.90  53.06  45.57      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
75    -0.46        2.04     6.51      X40      ATH          IT  
40     0.07        2.29    10.34      X40      BTT          IT  
42     0.20        1.44     7.39      X40      NTT        FMCG  
84    -0.16        2.08     5.91     X40N      ATH        FMCG  
1     -0.10        1.76     5.87       AR      NTT  ELECTRICAL

In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
7   ASIANTILES   137.00  7055.56    62.0       L-SC     7.19   78484.0   
20   COFFEEDAY    80.00   -45.67    49.0       L-SC    26.97   81024.0   
49      MASFIN   398.61   -19.60    48.0       H-SC     9.55   91380.0   
86   WHIRLPOOL  2270.00   -39.28    63.0       M-SC     3.86  102735.0   
46    KPIGREEN   731.05     7.10    67.0       H-SC     5.19  127789.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
7      -15326.0  91944.0       -2.67        -16.34  117.15  81.67    269.0   
20     -32525.0  72573.0        0.29        -28.64   89.57  35.27    268.0   
49      -6600.0  28200.0       -0.81         -6.74   30.86  22.05    152.0   
86      11236.0  67517.0       -2.12         12.28   65.72  86.07    223.0   
46       2492.0  56432.0       -0.30          1.99   44.16  47.03    141.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
7     -0.17        0.56    51.99       XR      NTT  CERAMICS  
20    -0.45        0.57    98.31       XR      NTT    HOTELS  
49    -0.23        0.65    32.83       XR      ATH   FINANCE  
86     0.17        0.73    49.19       XR      NTT  DURABLES  
46     0.04        0.90    60.00       MH      ATH     POWER

In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
62  SAMMAANCAP   326.00  -197.73    81.0       M-SC    24.60  169092.0   
79  UJJIVANSFB    60.00   122.87    69.0       H-SC    15.26  140508.0   
68    SONACOMS   806.63   -32.99    68.0       M-SC     9.63   83352.0   
5     ANGELONE  3033.00    16.33    70.0       X-SC     6.38  205820.0   
59   REPCOHOME   880.00   -55.29    72.0       H-SC     2.96  257273.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
62      19872.0  124316.0        7.73         13.32   73.52  96.62    208.0   
79      -1971.0   21498.0        0.27         -1.38   15.30  13.70    163.0   
68     -17807.0   57805.0       -0.37        -17.60   69.35  39.54    202.0   
5       14814.0   42893.0       -0.04          7.76   20.84  30.21    157.0   
59     -29232.0  286576.0        0.97        -10.20  111.39  89.82    134.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
62     0.16        1.20    82.99     XY25      NTT  FINANCE  
79    -0.09        0.99    65.84    OX40N      NTT    BANKS  
68    -0.31        0.59    17.54       AR      ATH     AUTO  
5      0.35        1.46    30.26     X40N      NTT  FINANCE  
59    -0.10        1.82    36.12     XY24      NTT  FINANCE

In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.41
1     25       43.78
2     50       74.73

In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.48
LC    32.68
MC    22.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.26
X40      16.02
XY25     12.29
XR       11.30
X40N      9.33
AR        7.91
OX40N     7.63
X200      1.76
X5K       1.50
SR        1.09
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.63
X-LC    21.87
X-MC    17.10
M-SC    12.74
M-LC     5.15
X-SC     4.64
H-LC     4.63
H-MC     3.85
M-MC     1.58
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.68         -3.95   37.87
IT                  12.71        -17.09   80.22
FINANCE             12.23         -9.43   59.04
MISC                 6.93        -17.39   82.44
BANKS                6.59         -6.85   63.05
PAINTS               5.64        -16.01   33.30
ELECTRICAL           5.30         -9.98   48.56
INSURANCE            3.89         -0.75   38.31
AUTO                 2.80        -44.92  105.40
FOOTWEAR             2.39        -30.66   83.04
TRAVEL               2.22        -43.15  114.83
AC                   2.21         -2.15   14.22
CERAMICS             2.19        -29.58   84.51
DURABLES             2.17        -18.91   44.69
BREWERIES            1.65         -0.40   21.25
REFINERIES           1.60          6.96    5.78
STEEL                1.58         -0.67   75.70
METALS               1.50          3.21   49.93
ENGINEERING          1.41        -21.21   76.65
CHEMICALS            1.32        -41.86  157.60
CEMENT               1.31        -28.31  110.62
JEWELLERY            1.29        -74.73  135.44
HEALTHCARE           1.20         -6.65   33.41
MINING               1.03         -3.46   26.69
ENTERTAINMENT        0.96        -42.21  127.03
POWER                0.90          1.95   44.16
PHARMA               0.64         -1.06   17.67
APPARELS             0.58         -2.04   26.51
HOTELS               0.57        -40.14   89.57
TEXTILES             0.50        -30.96   68.30

In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3294225.0     22
XR        1311802.0     14
AR        1178938.0      9
X40        782556.0     12
X40N       613642.0      8
OX40N      567819.0     10
XY25       446695.0      8
SR         267747.0      2
X5K        105785.0      1
X200        75155.0      1
MH          56432.0      1

In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        3377043.0     25
M-SC        1207628.0     15
X-MC        1119873.0     13
X-LC        1049499.0     13
X-SC         526524.0      6
H-MC         335395.0      3
H-LC         283826.0      3
M-LC         280357.0      4
L-SC         253105.0      3
M-MC         169162.0      1
L-MC          59569.0      1
L-LC          38815.0      1

In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1168754.0      6
           AR         845803.0      5
           XR         775220.0      7
M-SC       XY24       695963.0      6
X-MC       XY24       582600.0      4
X-LC       X40        451292.0      5
           XY24       308817.0      2
X-SC       X40N       284012.0      4
H-SC       SR         267747.0      2
           OX40N      263087.0      4
X-MC       X40        253754.0      6
X-LC       X40N       212633.0      3
H-LC       AR         208671.0      2
H-MC       XY24       179669.0      1
M-MC       XY24       169162.0      1
X-MC       XY25       166522.0      2
X-SC       XY24       165002.0      1
L-SC       XR         164517.0      2
M-SC       OX40N      147255.0      4
           AR         124464.0      2
           XY25       124316.0      1
X-MC       X40N       116997.0      1
M-SC       XR         115630.0      2
M-LC       XR         110029.0      1
           X5K        105785.0      1
L-SC       OX40N       88588.0      1
H-MC       XR          86837.0      1
X-SC       X40         77510.0      1
X-LC       XY25        76757.0      3
H-LC       X200        75155.0      1
H-MC       OX40N       68889.0      1
L-MC       XR          59569.0      1
H-SC       MH          56432.0      1
M-LC       XY25        40285.0      1
L-LC       XY25        38815.0      1
M-LC       XY24        24258.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
